In [493]:
import pandas as pd

In [494]:
hotels = pd.read_csv('data/hotels.csv')
hotels.head()

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671
1,130 134 Southampton Row Camden London WC1B 5AF...,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097
2,151 bis Rue de Rennes 6th arr 75006 Paris France,32,10/18/2016,8.9,Legend Saint Germain by Elegancia,China,No kettle in room,6,406,No Positive,0,14,7.5,"[' Leisure trip ', ' Solo traveler ', ' Modern...",289 day,48.845377,2.325643
3,216 Avenue Jean Jaures 19th arr 75019 Paris Fr...,34,9/22/2015,7.5,Mercure Paris 19 Philharmonie La Villette,United Kingdom,No Negative,0,607,Friendly staff quiet comfortable room spotles...,11,8,10.0,"[' Leisure trip ', ' Solo traveler ', ' Standa...",681 day,48.888697,2.394540
4,Molenwerf 1 1014 AG Amsterdam Netherlands,914,3/5/2016,8.5,Golden Tulip Amsterdam West,Poland,Torn sheets,4,7586,The staff was very friendly and helpful Break...,20,10,9.6,"[' Business trip ', ' Couple ', ' Standard Dou...",516 day,52.385601,4.847060


In [495]:
hotels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               386803 non-null  object 
 1   additional_number_of_scoring                386803 non-null  int64  
 2   review_date                                 386803 non-null  object 
 3   average_score                               386803 non-null  float64
 4   hotel_name                                  386803 non-null  object 
 5   reviewer_nationality                        386803 non-null  object 
 6   negative_review                             386803 non-null  object 
 7   review_total_negative_word_counts           386803 non-null  int64  
 8   total_number_of_reviews                     386803 non-null  int64  
 9   positive_review                             386803 non-null  object 
 

In [496]:
#object_columns = hotels.select_dtypes(include='object').columns
#hotels.drop(object_columns, axis=1, inplace=True)

In [497]:
#hotels.fillna(0, inplace=True)

In [498]:
X = hotels.drop('reviewer_score', axis=1)
y = hotels['reviewer_score']

In [499]:
X['review_date'] = pd.to_datetime(X['review_date']).dt.strftime('%Y/%m')

In [500]:
def rev_func(num_tags):
    num_tags = num_tags[2:-2]
    res = num_tags.strip().split(' \', \' ')
    return res

X['tags_n'] = X['tags'].apply(rev_func)



In [501]:
from collections import Counter
tag_dict = {}
for tags in X['tags_n']:
    for tag in tags:
        tag_dict[tag] = tag_dict.setdefault(tag, 0) + 1
top_tags = Counter(tag_dict)
top_tags = list(map(lambda l:l[0], list(top_tags.most_common(5))))

def check_top_tags(tags_list):
    for tag in tags_list:
        if tag in top_tags:
            return 1
        else:
            return 0
    
X['top_tag'] = X['tags_n'].apply(check_top_tags)
X['one_night'] = X['tags_n'].apply(lambda l:1 if 'Stayed 1 night' in l else 0)
X['two_nights'] = X['tags_n'].apply(lambda l:1 if 'Stayed 2 nights' in l else 0)

In [502]:
X.drop('tags_n', axis=1, inplace=True)

In [503]:
num_cols = ['additional_number_of_scoring',
    'review_total_negative_word_counts', 'total_number_of_reviews', 
    'review_total_positive_word_counts', 'total_number_of_reviews_reviewer_has_given']
all_cat_cols = ['average_score', 'lat', 'hotel_address', 'hotel_name', 'reviewer_nationality', 
            'negative_review', 'positive_review', 'tags', 'lng', 'review_date', 'days_since_review']
cat_cols = ['average_score', 'hotel_name', 'reviewer_nationality', 'review_date']


In [504]:
import category_encoders as ce
ord_encoder = ce.OrdinalEncoder(cols=cat_cols)
X = ord_encoder.fit_transform(X)

In [505]:
#from sklearn.feature_selection import chi2
#imp_cat = pd.Series(chi2(X[cat_cols], y)[0], index=cat_cols).sort_values()
#imp_cat.plot(kind='barh')

In [506]:
object_columns = X.select_dtypes(include='object').columns
X.drop(object_columns, axis=1, inplace=True)
X.drop(['lat', 'lng'], axis=1, inplace=True)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 12 columns):
 #   Column                                      Non-Null Count   Dtype
---  ------                                      --------------   -----
 0   additional_number_of_scoring                386803 non-null  int64
 1   review_date                                 386803 non-null  int64
 2   average_score                               386803 non-null  int64
 3   hotel_name                                  386803 non-null  int64
 4   reviewer_nationality                        386803 non-null  int64
 5   review_total_negative_word_counts           386803 non-null  int64
 6   total_number_of_reviews                     386803 non-null  int64
 7   review_total_positive_word_counts           386803 non-null  int64
 8   total_number_of_reviews_reviewer_has_given  386803 non-null  int64
 9   top_tag                                     386803 non-null  int64
 10  one_night           

In [507]:
X.shape

(386803, 12)

In [508]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [509]:
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

regr = RandomForestRegressor(n_estimators=100)
regr.fit(X_train, y_train)

y_pred = regr.predict(X_test)

In [510]:
print('MAPE: ', metrics.mean_absolute_percentage_error(y_test, y_pred))

MAPE:  0.1385825526005986


# MAPE Record
1. Standard code - 0.1413185433974396
2. ['average_score', 'lat', 'hotel_name', 'reviewer_nationality', 'lng'] coded - 0.140
3. ['average_score', 'hotel_name', 'reviewer_nationality', 'review_date'] coded - 0.13900343230788084
4. Top tags added - 0.13884760809561972
5. 1/2 nights added - 0.1385825526005986